## Implementing Chris Doenlen's 'Bot Or Not' Python Module

Everything from `twitter_funcs.py` was cloned from Chris' [repository](https://github.com/scrapfishies/twitter-bot-detection).

I will use this to label each user as 'bot' (boolean 1/0).

In [2]:
import pandas as pd
import numpy as np
from copy import deepcopy
from twitter_funcs import *

# Imports
import os
import numpy as np
import pandas as pd

import pickle
import tweepy

from datetime import datetime
from secrets import api_secret_key, api_key, bearer_token
import re
import time
import csv

In [3]:
data = pd.read_pickle("pickle/n2_tokenized_eff.pick")

In [4]:
data.head(3)

,id,date,time,user_id,username,hashtags,trump,biden,original,tweet,num_tokens
181142,1323379284434669568,2020-11-02,21:39:43,2820503362,artistacriseida,[],False,True,All these articles showing that Biden is in th...,article showing lead ignore govote map showing...,16
0,1323414585995526144,2020-11-02,23:59:59,1312487180258820096,annapieters17,[],False,True,@FoxNews Lady Gaga’s a nobody. Can’t figure ou...,foxnews lady nobody figure life nobody help ga...,10
4,1323414585232293888,2020-11-02,23:59:59,2335763630,kylechwatt,[],False,True,@The_Grupp “It is purely a fortuity that this ...,purely fortuity mass casualty event history ch...,8


I'm getting a rate limit error. according to twitter site, I can lookup 300 users per 15 minutes. Let's try that.

In [5]:
def chunks(user_ids, n):
    """Yield successive n-sized chunks from user_ids (iterable)."""
    lst = list(user_ids)
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
    


Here's where we'll implement Chris Doenlen's 'Bot or Not' model.

In [7]:
with open("pickle/bot_model.pick", "rb") as read_file:
    xgb_model = pickle.load(read_file)

In [ ]:
# time.sleep(60*15)

In [ ]:
# verifieds = {}
# bot_probas = {}

# all_users = list(data.user_id.unique())
# print(f"Total number of users to scrape: {len(all_users)}")
# exist = pd.read_csv("../data/user_stats.csv")
# exist_users = list(exist.user_id.unique())
# print(f"Number of users already scraped: {len(exist_users)}")

# user_ids = []
# for user in exist_users:
#     if user in all_users:
#         continue
#     else:
#         user_ids.append(user)
    
# print(f"Preparing to identify bots for {len(user_ids)} users...")

# user_id_chunks = list(chunks(user_ids, n=300))

# # now get stats for new users

# csv_file = open("../data/user_stats.csv", "a")
# csv_writer = csv.writer(csv_file)
# csv_writer.writerow(["user_id", "bot_proba", "verified"])
# for chunk in user_id_chunks:
#     print(f"Preparing chunk. Num users: {len(chunk)}")
#     for user_id in chunk:
#         print(f"Preparing user '{user_id}'")
        
        
#         auth = tweepy.OAuthHandler(api_key, api_secret_key)
#         api = tweepy.API(auth)
        
#         try: # Gather features for bot/not bot model
#             # Get user information from screen name
#             user = api.get_user(user_id)

#             # account features to return for predicton
#             account_age_days = (datetime.now() - user.created_at).days
#             verified = user.verified # will also use this in our data
#             geo_enabled = user.geo_enabled
#             default_profile = user.default_profile
#             default_profile_image = user.default_profile_image
#             favourites_count = user.favourites_count
#             followers_count = user.followers_count
#             friends_count = user.friends_count
#             statuses_count = user.statuses_count
#             average_tweets_per_day = np.round(statuses_count / account_age_days, 3)

#             # manufactured features
#             hour_created = int(user.created_at.strftime("%H"))
#             network = np.round(np.log(1 + friends_count) * np.log(1 + followers_count), 3)
#             tweet_to_followers = np.round(
#                 np.log(1 + statuses_count) * np.log(1 + followers_count), 3
#             )
#             follower_acq_rate = np.round(
#                 np.log(1 + (followers_count / account_age_days)), 3
#             )
#             friends_acq_rate = np.round(np.log(1 + (friends_count / account_age_days)), 3)

#             # organizing list to be returned
#             account_features = [
#                 verified, hour_created,geo_enabled,default_profile,default_profile_image,favourites_count,
#                 followers_count,friends_count,statuses_count,average_tweets_per_day,network,tweet_to_followers,
#                 follower_acq_rate,friends_acq_rate]

#             if account_features == np.nan:
#                 proba = np.nan
#                 verified = np.nan
#                 csv_writer.writerow([user_id, proba, verified])
#                 continue

#             else:
#                 user_m = np.matrix(account_features)
#                 proba = np.round(xgb_model.predict_proba(user_m)[:, 1][0] * 100, 2)
#                 verified = account_features[0]
#                 csv_writer.writerow([user_id, proba, verified])

#         except:
#             print(f'error encountered, skipping user {user_id}')
#             proba = np.nan
#             verified = np.nan
        
#             csv_writer.writerow([user_id, proba, verified])
#     print("Chunk complete. Waiting 15 minutes.")
#     time.sleep(15*60+1)

# csv_file.close()

In [ ]:
# one-off copy
# time.sleep(120)
verifieds = {}
bot_probas = {}

user_ids = list(pd.read_pickle("pickle/grab_users.pick"))
    

user_id_chunks = list(chunks(user_ids, n=300))

# now get stats for new users

csv_file = open("../data/user_stats.csv", "a")
csv_writer = csv.writer(csv_file)
csv_writer.writerow(["user_id", "bot_proba", "verified"])
for chunk in user_id_chunks:
    print(f"Preparing chunk. Num users: {len(chunk)}")
    for user_id in chunk:
        print(f"Preparing user '{user_id}'")
        
        
        auth = tweepy.OAuthHandler(api_key, api_secret_key)
        api = tweepy.API(auth)
        
        try: # Gather features for bot/not bot model
            # Get user information from screen name
            user = api.get_user(user_id)

            # account features to return for predicton
            account_age_days = (datetime.now() - user.created_at).days
            verified = user.verified # will also use this in our data
            geo_enabled = user.geo_enabled
            default_profile = user.default_profile
            default_profile_image = user.default_profile_image
            favourites_count = user.favourites_count
            followers_count = user.followers_count
            friends_count = user.friends_count
            statuses_count = user.statuses_count
            average_tweets_per_day = np.round(statuses_count / account_age_days, 3)

            # manufactured features
            hour_created = int(user.created_at.strftime("%H"))
            network = np.round(np.log(1 + friends_count) * np.log(1 + followers_count), 3)
            tweet_to_followers = np.round(
                np.log(1 + statuses_count) * np.log(1 + followers_count), 3
            )
            follower_acq_rate = np.round(
                np.log(1 + (followers_count / account_age_days)), 3
            )
            friends_acq_rate = np.round(np.log(1 + (friends_count / account_age_days)), 3)

            # organizing list to be returned
            account_features = [
                verified, hour_created,geo_enabled,default_profile,default_profile_image,favourites_count,
                followers_count,friends_count,statuses_count,average_tweets_per_day,network,tweet_to_followers,
                follower_acq_rate,friends_acq_rate]

            if account_features == np.nan:
                proba = np.nan
                verified = np.nan
                csv_writer.writerow([user_id, proba, verified])
                continue

            else:
                user_m = np.matrix(account_features)
                proba = np.round(xgb_model.predict_proba(user_m)[:, 1][0] * 100, 2)
                verified = account_features[0]
                csv_writer.writerow([user_id, proba, verified])

        except:
            print(f'error encountered, skipping user {user_id}')
            proba = np.nan
            verified = np.nan
        
            csv_writer.writerow([user_id, proba, verified])
    print("Chunk complete. Waiting 15 minutes.")
    time.sleep(15*60+1)

csv_file.close()

Preparing chunk. Num users: 300
Preparing user '1143289465387081732'
Preparing user '782603330'
Preparing user '777679662310907904'
Preparing user '1408230732'
Preparing user '20406676'
Preparing user '1304172941391863817'
Preparing user '51827732'
Preparing user '3937940121'
Preparing user '8500962'
Preparing user '570097148'
Preparing user '104246758'
Preparing user '1043423847708590081'
Preparing user '31571443'
Preparing user '547892272'
Preparing user '160364793'
Preparing user '1251323699451564035'
Preparing user '20824974'
Preparing user '51183401'
Preparing user '1649886427'
Preparing user '15145034'
Preparing user '112557979'
Preparing user '3304837284'
Preparing user '832841680135081984'
Preparing user '1241019363525677057'
Preparing user '1255552876073680903'
Preparing user '1287515682641412096'
Preparing user '817761228688916480'
Preparing user '1239930170800107521'
Preparing user '283350568'
Preparing user '850046670'
Preparing user '728930499029172224'
Preparing user '444

Preparing user '1190079278857281539'
Preparing user '207656440'
Preparing user '912135002782969857'
Preparing user '212795909'
Preparing user '2362868665'
Preparing user '72592074'
Preparing user '585764087'
Preparing user '1072212454686515209'
Preparing user '1102655017042038785'
Preparing user '91808599'
Preparing user '1176325494259798017'
Preparing user '1289930821357445132'
Preparing user '34283650'
Preparing user '67035595'
Preparing user '1375297094'
Preparing user '1230089817418895360'
Preparing user '993215081256181760'
Preparing user '26965771'
Preparing user '1220666675072204801'
Preparing user '41964897'
Preparing user '219174241'
Preparing user '1016066208'
Preparing user '529718790'
Preparing user '1041056512582533121'
Preparing user '124275780'
Preparing user '1180983441044770817'
Preparing user '3167620869'
Preparing user '1234187035'
Preparing user '1309975396943880192'
Preparing user '17773224'
Preparing user '94823672'
Preparing user '26928006'
Preparing user '802581

In [ ]:
bot_probas

In [ ]:
verifieds

In [ ]:
data['verified'] = data['user_id'].map(verifieds)
data['bot_proba'] = data['user_id'].map(bot_probas)
